In [2]:
import tweepy
import webbrowser
import winsound
import time
import csv
import datetime
import re
import pandas as pd
import os.path
from datetime import datetime as dt

frequency = 2500  # Set Frequency To 2500 Hertz
duration = 200  # Set Duration To 1000 ms == 1 second

consumer_key = ""
consumer_secret = ""

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
webbrowser.open(auth.get_authorization_url())
print('Enter id:')
verifier = input()
auth.get_access_token(verifier)

api = tweepy.API(auth, wait_on_rate_limit=True)

Enter id:
7697742


In [5]:
path = 'C:/Users/Maxim/Nero/'
columns = ['id', 'sentiment', 'review', 'datati', 'usid', 'followers']


class export_to_tsv:
    def __init__(self, path, filename):
        columns = ['id', 'sentiment', 'review', 'datati', 'usid', 'followers']
        self.path = path
        self.sent = None
        self.filename = filename
        self.data = pd.DataFrame(columns=columns)
        if (os.path.exists(self.path+self.filename) is False):
            self.data.to_csv(path_or_buf=self.path+self.filename,
                             mode='a', sep="\t")

    def setSentiment(self, sent):
        if sent == 'p':
            self.sent = 1
            return
        if sent == 'n':
            self.sent = 0
            return
        else:
            print('incorrect sentiment: ')
            print('write "p"for positive or "n" for negative')
            return

    def add(self, tweet):
        if(self.sent is None):
            print('!!set sentiment of class')
            return
        s = pd.Series({'id': tweet.id, 'sentiment': self.sent,
                       'review': tweet.text,
                       'datati': tweet.created_at,
                       'usid': tweet.user.id,
                       'followers': tweet.user.followers_count})
        self.data = self.data.append(s, ignore_index=True)
        return self.data.shape

    def output(self):
        self.data.to_csv(path_or_buf=self.path+self.filename,
                         mode='a', sep="\t", header=False, index=False)

    def remove(self):
        self.data = pd.DataFrame(columns=columns)


In [6]:
def isRetweet(text):
    result = re.match(r'RT', text)
    if(result):
        return True
    else:
        return False


def filter(item):
    item = item.replace("’", "'")
    item = item.replace("”", '"')
    item = item.replace("“", '"')
    item = item.replace("…", ' ')
    item = item.replace("&amp;", ' and ')
    item = re.sub("([…:;|])", " ", item)
    item = item.replace("—", '-')
    item = item.replace("\n", ' ')
    item = item.replace("\t", ' ')
    #item = item.replace("([a-z])\\1+", r"\1")
    item = re.sub("([a-zA-Z])\\1+\\1+", r"\1", item)
    item = re.sub("([!?\"'])\\1+", r"\1", item)
    item = re.sub("([\.])\\1+", r" \1\1\1 ", item)
    item = re.sub("([!?])", r" \1 ", item)
    item = re.sub("([^\w\.,\-'\"!?])", r" ", item)
    item = ' '.join(item.split())
    #print(item)

    return item


def cleaner_text(text, emoji_del=True, links_del=True, url_del=True):
    #regex for url
    urlregex = re.compile(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*')
    #regex for emoji
    emojiregex = re.compile(r'([\U0001F601-\U0001F64F]|[\U0001F900-\U0001F9FF]|[\U0001F680-\U0001F6FF])')
    #regx for links
    linksregex = re.compile(r'@\w+')
    #regex for hashtags
    hashregex = re.compile(r'#\w+')

    if(url_del):
        text = urlregex.sub('', text)
    if(links_del):
        text = linksregex.sub('', text)
    if(emoji_del):
        text = emojiregex.sub('', text)
    text = hashregex.sub('', text)
    text = filter(text)
    return text


In [7]:
limit_iter=0
def test_rate_limit(api, wait=True, buffer=.1):
    #Get the number of remaining requests
    remaining = int(api.last_response.headers.get('x-rate-limit-remaining'))
    #Check if we have reached the limit
    limit_iter+=1
    if remaining == 0:
        limit = int(api.last_response.headers.get('x-rate-limit-limit'))
        reset = int(api.last_response.headers.get('x-rate-limit-reset'))
        #Parse the UTC time
        reset = dt.fromtimestamp(reset)
        #Let the user know we have reached the rate limit
        print ("0 of {} requests remaining until {}.".format(limit, reset))

        if wait:
            #Determine the delay and sleep
            delay = (reset - dt.now()).total_seconds() + buffer
            print ("Sleeping for {}s...".format(delay))
            limit_iter+=1
            time.sleep(delay)
            #We have waited for the rate limit reset. OK to proceed.
            return True
        else:
            #We have reached the rate limit. The user needs to handle the rate limit manually.
            print('rate lol')
            return False


    #We have not reached the rate limit
    return True

In [8]:
#from tqdm import tqdm
from IPython.display import clear_output

import os
COUNT = 200
MIN_LEN_TWEET = 40
#😭😂


def search(symbol, sent, filename, limit=20000, max_id=None):
    latest_tweet = dt.now()
    total_count = 0
    count_row = 0
    iter_i = 0
    if(max_id is None):
        first_tweet = api.search(symbol, count=1, lang='en',
                                 result_type='recent')
        max_id = first_tweet[0].id
    print('lets go!!')
    tsv = export_to_tsv(path, filename)
    tsv.setSentiment(sent)
    #tsv.setSentiment('p')
    while(count_row < limit):
        #test_rate_limit(api, wait=True, buffer=.1)
        public_tweets = api.search(symbol, count=COUNT, lang='en',
                                   result_type='recent', max_id=max_id)
        total_count += public_tweets.count
        for t in public_tweets:
            if(t.id < max_id):
                max_id = t.id-1
            if(latest_tweet > t.created_at):
                latest_tweet = t.created_at
            isRT = isRetweet(t.text)
            if(isRT):
                #print('its retweet')
                continue
            clean_text = cleaner_text(t.text)
            if(len(clean_text) < MIN_LEN_TWEET):
                #print('tweet low then ',MIN_LEN_TWEET)
                continue
            t.text = clean_text
            data_count = tsv.add(t)
            count_row += 1
            if(data_count[0] == 200):
                tsv.output()
                tsv.remove()
                #print('save 200')
        iter_i += 1
        clear_output(wait=True)
        print('symbol: ', symbol)
        print('записей в бд:', count_row)
        print('Количество итераций:%d ' % (iter_i))
        print('Всего обработанных твитов:', total_count)
        print('последний твит ', latest_tweet.strftime("%Y-%m-%d %H:%M:%S"))
        print('max_id: ', max_id)
        print('--------------------------------')


In [9]:
#search(symbol='😂',sent='p',filename='positive_emoji_2_07-02-18.tsv',limit=75000, max_id=961195454611574783)
#search(symbol='😭',sent='n',filename='negative_emoji_2_07-02-18.tsv',limit=75000, max_id=960894076949155841)
#search(symbol='😭',sent='n',filename='negative_emoji_2_07-02-18.tsv',limit=75000)
search(symbol='bitcoin', sent='n', filename='bitcoin8.tsv', limit=400000)


symbol:  bitcoin
записей в бд: 216765
Количество итераций:6765 
Всего обработанных твитов: 676500
последний твит  2018-05-06 06:05:48
max_id:  993008913007173631
--------------------------------


TweepError: [{'message': 'Internal error', 'code': 131}]

In [ ]:
993170404989685762